In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from pandas_profiling import ProfileReport

In [2]:
c5 = pd.read_csv('incidentes-viales-c5.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'incidentes-viales-c5.csv'

In [ ]:
c5.head()

In [ ]:
len(c5.columns)

In [ ]:
len(c5)

In [ ]:
c5.nunique()

In [ ]:
c5.dtypes

In [ ]:
#hay que hacer varios cambios en el tipo

In [ ]:
#variables numéricas
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
nc5 = c5.select_dtypes(include=numerics)
print('Son ' + str(nc5.shape[1]) + ' variables numericas')

In [ ]:
profile = ProfileReport(c5, title = "Profiling")
profile.to_file("./data-profiling.html")
profile.to_widgets()

In [ ]:
def data_profiling(dataframe):
    """
    Crea el data profiling de variables numericas y categoricas
    """
    
    # Primero asignamos un data frame con los nombres de las columnas que saldran
    df_num = pd.DataFrame(columns = ['tipo_de_dato','observaciones','media','cuantil_25',
                                   'cuantil_50','cuantil_75','valor_max','valor_min',
                                   'observaciones_unicas','top_1','top_2','top_3','numero_de_nan'])
    df_cat = pd.DataFrame(columns = ['n_de_categorias','moda','top_1','top_2','top_3','numero_de_nan'])
    
    
    # Hacemos un loop para que itere sobre las columnas del data frame
    for columna in dataframe:
        
        # Condicional para datos numericos
        if pd.api.types.is_numeric_dtype(dataframe[columna]):
            # Estadisticos
            tipo_de_dato = dataframe[columna].dtype 
            obs = len(dataframe[columna])
            std = dataframe[columna].std()
            media = dataframe[columna].mean()
            cuantil_25 = dataframe[columna].quantile(.25)
            cuantil_50 = dataframe[columna].quantile(.5)
            cuantil_75 = dataframe[columna].quantile(.75)
            maximo = dataframe[columna].max()
            minimo = dataframe[columna].min()
            obs_unic = dataframe[columna].nunique()
            num_de_nan = dataframe[columna].isna().sum()
            # Top 3
            if dataframe[columna].nunique() >= 3:
                top_3= dataframe.groupby([columna])[columna].count().sort_values(ascending = False).head(3)
                top_1 = top_3.index[0]
                top_2 = top_3.index[1]
                top_3 = top_3.index[2]
            else:
                top_3 = dataframe.groupby([columna])[columna].count().sort_values(ascending = False).head(3)
                top_1 = top_3.index[0]
                top_2 = top_3.index[0]
                top_3 = top_3.index[0]
                
            # Asignamos los estadisticos a las columnas
            datos = {'tipo_de_dato':tipo_de_dato,'observaciones':obs,'media':media,
                    'cuantil_25':cuantil_25,'cuantil_50':cuantil_50,'cuantil_75':cuantil_75,
                    'valor_max':maximo,'valor_min':minimo,'observaciones_unicas':obs_unic,
                    'top_1':top_1,'top_2':top_2,'top_3':top_3,'numero_de_nan':num_de_nan}
            
            # Los vamos agregando a un nuevo data frame 
            df1 = pd.DataFrame(datos, index = [columna])
            df_num = df_num.append(df1)
        
        # Condicional para los categoricos    
        else:
            # Estadisticos
            num_cat = dataframe[columna].nunique()
            moda = dataframe[columna].mode(dropna=True)[0]
            num_de_nan = dataframe[columna].isna().sum()
            # Top 3
            if dataframe[columna].nunique() >= 3:
                top_3= dataframe.groupby([columna])[columna].count().sort_values(ascending = False).head(3)
                top_1 = top_3.index[0]
                top_2 = top_3.index[1]
                top_3 = top_3.index[2]
            else:
                top_3 = dataframe.groupby([columna])[columna].count().sort_values(ascending = False).head(3)
                top_1 = top_3.index[0]
                top_2 = top_3.index[0]
                top_3 = top_3.index[0]
                
            # Asignamos estadisticos a columnas
            datos = {'n_de_categorias':num_cat,'moda':moda,'top_1':top_1,'top_2':top_2,
                     'top_3':top_3,'numero_de_nan':num_de_nan}
            
            # Los vamos agregando a un nuevo data frame
            df1 = pd.DataFrame(datos, index = [columna])
            df_cat = df_cat.append(df1)
            
    # Regresa dos bases una para las variables numericas y otro para variables categoricas
    return df_num, df_cat

In [ ]:
numerico, categorico = data_profiling(c5)

In [ ]:
display(numerico)

In [ ]:
# Número de categorías, moda, top 3 y valores faltantes
display(categorico)

In [ ]:
# Valor de las categorías
c5.año_cierre.unique()

In [ ]:
c5.mes.unique()

In [ ]:
c5.dia_semana.unique()

In [ ]:
c5.codigo_cierre.unique()

In [ ]:
c5.delegacion_inicio.unique()

In [ ]:
c5.incidente_c4.unique()

In [ ]:
c5.clas_con_f_alarma.unique()

In [ ]:
c5.tipo_entrada.unique()

In [ ]:
c5.delegacion_cierre.unique()

In [ ]:
# Número de observaciones por categoría y proporción de observaciones por categoría. 

def num_prop(df,column_data):
    # Número de observaciones por categoría
    num=pd.value_counts(df[column_data]) 
    # Proporción de observaciones por categoría
    prop=100 * df[column_data].value_counts() / len(df[column_data])
    # Unión de columnas en una y renombramiento
    columnas_num_prop = round(pd.concat([num, prop], 
                                        keys=['num_obs_cat', 'prop_obs_cat'],
                                        axis=1),2)
    
    return columnas_num_prop

In [ ]:
num_prop(c5,'mes')

In [ ]:
num_prop(c5,delegacion_inicio)

In [ ]:
c5.groupby(['delegacion_inicio']).count()

In [ ]:
list(c5.columns)

In [ ]:
geo = c5.geopoint.str.split(",", n=1, expand = True)
geo

# GEDA

In [ ]:
consu_alca_tot = pd.DataFrame(c5,
                      columns=['delegacion_inicio',
                               'incidente_c4',
                               'clas_con_f_alarma',
                               'tipo_entrada'])\
.groupby(['delegacion_inicio'], as_index=False).sum()#\
#.sort_values(by=['consumo_prom'], ascending=False)\
#.melt(id_vars='delegacion_inicio')



In [ ]:
sns.set(style='whitegrid',rc={'figure.figsize':(8,11)},palette='crest')
plt1 = sns.barplot(x="value", y="alcaldia", 
            data=consu_alca_tot, hue = 'variable')
plt1.set(xlabel='Consumo Agua m3')
plt.show()

# Cambio de variables

In [ ]:
c5.dtypes

In [ ]:
c5['folio'] = c5.folio.astype('str')
#c5['fecha_creacion'] = c5.fecha_creacion.astype('date')
#c5['hora_creacion'] = c5.hora_creacion.astype('time')
c5['dia_semana'] = c5.dia_semana.astype('category')
c5['codigo_cierre'] = c5.codigo_cierre.astype('str')
#c5['fecha_cierre'] = c5.fecha_cierre.astype('date')
#c5['mes_cierre'] = c5.mes_cierre.astype('int')
#c5['hora_cierre'] = c5.hora_cierre.astype('time')
c5['delegacion_inicio'] = c5.delegacion_inicio.astype('category')
c5['incidente_c4'] = c5.incidente_c4.astype('str')
c5['clas_con_f_alarma'] = c5.clas_con_f_alarma.astype('str')
c5['tipo_entrada'] = c5.tipo_entrada.astype('category')
c5['delegacion_cierre'] = c5.delegacion_cierre.astype('category')
#eliminar geopoint

# Imputación

#Variables a imputar

latitud

longitud

delegacion_inicio

delegacion_cierre

# Graficas Diego

Mapas

In [ ]:
data.groupby('delegacion_inicio')['folio'].nunique()
data_map=data[data.delegacion_inicio=="IZTAPALAPA"]

In [ ]:
fig = px.scatter_mapbox(data_map, lat="latitud", lon="longitud",color="codigo_cierre",color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=3)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
data=data.assign(count=1)

In [ ]:
pivot_1=data[['count','dia_semana','clas_con_f_alarma']].groupby(['clas_con_f_alarma','dia_semana']).sum().reset_index()

In [ ]:
sns.barplot(x="count", y="clas_con_f_alarma", data=pivot_1)